In [1]:
from urllib.request import urlretrieve
from os.path import isfile
from tqdm import tqdm
import collections
import json


## NUMPY IMPORTS
import pickle
import numpy as np
import math
import pandas as pd
from scipy.misc import imread

# TENSORFLOW
import tensorflow as tf
tf.python.control_flow_ops = tf


## KERAS
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout,Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.activations import relu, softmax
from keras.utils import np_utils


from sklearn.utils import shuffle
import matplotlib.pyplot as plt

#  having opencv 
import cv2


Using TensorFlow backend.


In [2]:
# constants
dataFolderPath = "data2/"
imageFolderPath = "data2/"

In [3]:
df = pd.read_csv(dataFolderPath+"driving_log.csv")

### preprocessing the image 

In [4]:
new_size_col,new_size_row = 64, 64

def trans_image(image,steer,trans_range):
    # Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    steer_ang = steer + tr_x/trans_range*2*.2
    tr_y = 40*np.random.uniform()-40/2
    #tr_y = 0
    cols = 320
    rows =160
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])
    image_tr = cv2.warpAffine(image,Trans_M,(cols,rows))
    
    return image_tr,steer_ang

def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    #print(random_bright)
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def add_random_shadow(image):
    top_y = 320*np.random.uniform()
    top_x = 0
    bot_x = 160
    bot_y = 320*np.random.uniform()
    image_hls = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    shadow_mask = 0*image_hls[:,:,1]
    X_m = np.mgrid[0:image.shape[0],0:image.shape[1]][0]
    Y_m = np.mgrid[0:image.shape[0],0:image.shape[1]][1]
    shadow_mask[((X_m-top_x)*(bot_y-top_y) -(bot_x - top_x)*(Y_m-top_y) >=0)]=1
    #random_bright = .25+.7*np.random.uniform()
    if np.random.randint(2)==1:
        random_bright = .5
        cond1 = shadow_mask==1
        cond0 = shadow_mask==0
        if np.random.randint(2)==1:
            image_hls[:,:,1][cond1] = image_hls[:,:,1][cond1]*random_bright
        else:
            image_hls[:,:,1][cond0] = image_hls[:,:,1][cond0]*random_bright    
    image = cv2.cvtColor(image_hls,cv2.COLOR_HLS2RGB)
    return image


def preprocessImage(image):
#     image = cv2.imread(imageFolderPath+imagePath)
#     image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    shape = image.shape
    # note: numpy arrays are (row, col)!
    image = image[math.floor(shape[0]/5):shape[0]-25, 0:shape[1]]
    image = cv2.resize(image,(new_size_col,new_size_row),         interpolation=cv2.INTER_AREA)    
#     image= cv2.resize(image,None,fx=.5, fy=.5, interpolation = cv2.INTER_AREA)

    #image = image/255.-.5
    return image

def preprocess_image_file_train(line_data):
    i_lrc = np.random.randint(3)
    if (i_lrc == 0):
        path_file = line_data['left'][0].strip()
        shift_ang = .25
    if (i_lrc == 1):
        path_file = line_data['center'][0].strip()
        shift_ang = 0.
    if (i_lrc == 2):
        path_file = line_data['right'][0].strip()
        shift_ang = -.25
    y_steer = line_data['steering'][0] + shift_ang
    image = cv2.imread(imageFolderPath+path_file)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
#     image,y_steer,tr_x = trans_image(image,y_steer,100)
    image = augment_brightness_camera_images(image)
    image = preprocessImage(image)
    image = np.array(image)
    ind_flip = np.random.randint(2)
    if ind_flip==0:
        image = cv2.flip(image,1)
        y_steer = -y_steer
    
    return image,y_steer




In [5]:
# df.iloc[:,0]
print(df.iloc[[0]])
preprocess_image_file_train(df.iloc[[0]])

                                   center  \
0  IMG/center_2017_01_29_22_27_17_218.jpg   

                                    left  \
0   IMG/left_2017_01_29_22_27_17_218.jpg   

                                    right  steering  throttle  brake    speed  
0   IMG/right_2017_01_29_22_27_17_218.jpg       0.0       0.0      0  0.00008  


(array([[[189, 193,  43],
         [175, 171,  54],
         [204, 196,  80],
         ..., 
         [ 46,  52,  46],
         [ 94,  88,  53],
         [ 46,  53,  26]],
 
        [[192, 197,  55],
         [179, 177,  66],
         [197, 191,  94],
         ..., 
         [ 39,  44,  33],
         [120, 116,  77],
         [ 40,  47,  29]],
 
        [[191, 196,  64],
         [175, 177,  71],
         [152, 149,  87],
         ..., 
         [ 69,  72,  50],
         [121, 118,  70],
         [ 32,  42,  32]],
 
        ..., 
        [[ 12,  11,   9],
         [ 36,  35,  21],
         [ 90,  95,  77],
         ..., 
         [115, 117, 103],
         [115, 117, 104],
         [ 99, 101,  87]],
 
        [[ 18,  18,  16],
         [ 85,  82,  75],
         [ 93, 106,  72],
         ..., 
         [133, 135, 121],
         [127, 129, 116],
         [114, 116, 102]],
 
        [[ 61,  57,  52],
         [101,  79, 101],
         [154, 142,  85],
         ..., 
         [106, 107,  94

In [6]:
# #  getting the xdata in the table set

# x_data=np.array([preprocessImage(imagePath) for imagePath in df.iloc[:,0]])
# print(np.shape(x_data))


In [7]:
# plt.imshow(x_data[758])
# plt.show()

In [8]:
## getting the ydata
# "{0:.2f}".format(round(data,2))
y_data = np.array([ data for data in df.iloc[:,3] ])
print(np.shape(y_data))

(5491,)


In [9]:
# # TODO: One Hot encode the labels to the variable y_one_hot
# from sklearn.preprocessing import LabelBinarizer
# label_binarizer = LabelBinarizer()
# y_one_hot = label_binarizer.fit_transform(y_data)


In [10]:
# print(np.shape(newXData))


In [11]:
# # here i want to visualize the data that is used
# sampleImage = imread(df.get_value(29,0))
# print (np.shape(sampleImage))
# # plt.axis("off")
# plt.imshow(sampleImage)
# # plt.show()


In [12]:
# preprocess the data
# from sklearn.preprocessing import LabelBinarizer
# label_binarizer = LabelBinarizer()
# y_one_hot = label_binarizer.fit_transform(y_data)


In [13]:
# this is nvidia model

In [14]:
# # TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# model = Sequential()
# # model.add(Convolution2D(32, 2, 2, input_shape=(32, 32, 3)))
# # model.add(Reshape(input_shape + (1, ), input_shape=input_shape))

# # model.add(Convolution2D(32, 3,3, input_shape=(160, 320, 3)))
# model.add(Convolution2D(24, 5, 5,input_shape=(80, 160, 3)))
# model.add(Convolution2D(36, 5, 5))
# model.add(Convolution2D(48, 5, 5))
# model.add(Convolution2D(64, 3, 3))
# # model.add(Convolution2D(48, 5, 5))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))

# # model.add(Convolution2D(64, 3, 3))
# # model.add(Convolution2D(64, 3, 3))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))


# # model.add(Convolution2D(128, 3, 3))
# # model.add(Convolution2D(128, 3, 3))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(1164))
# model.add(Activation('relu'))
# model.add(Dense(100))
# model.add(Activation('relu'))
# model.add(Dense(50))
# model.add(Activation('relu'))
# model.add(Dense(10))
# # model.add(Dense(len(classes)))
# # model.add(Activation('relu'))
# model.add(Activation('softmax'))

# # model.add(Dense(1164))
# # # model.add(Dense(100))
# # # model.add(Dense(50))
# # # model.add(Dense(10))

# # model.add(Activation('relu'))
# # model.add(Dense(100))
# # model.add(Activation('relu'))
# # model.add(Dense(50))
# # model.add(Activation('relu'))
# # model.add(Dense(10))


In [15]:
# comman ai way
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D

# ch, row, col = 3, 160, 320  # camera format
ch, row, col = 3, 64, 64  # camera format
model = Sequential()
# # model.add(Lambda(lambda x: x/127.5 - 1.,
# #           input_shape=(ch, row, col),
# #           output_shape=(ch, row, col)))
model.add(Convolution2D(16, 8, 8, input_shape=(64, 64, 3)))
model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
model.add(ELU())
model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(ELU())
model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(Flatten())
model.add(Dropout(.2))
model.add(ELU())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(1))




# model.add(Convolution2D(32, 3, 3,input_shape=(64, 64, 3)))
# model.add(Convolution2D(32, 3, 3))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Convolution2D(64, 3, 3))
# model.add(Convolution2D(64, 3, 3))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))


# model.add(Convolution2D(128, 3, 3))
# model.add(Convolution2D(128, 3, 3))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Dropout(.5))
# model.add(Activation('relu'))
# model.add(Dense(1))
# # model.add(Activation('softmax'))



In [16]:
val_size = 1
pr_threashold = 1
batch_size = 256

def generate_train_from_PD_batch(data,batch_size = 32):
    
    batch_images = np.zeros((batch_size, new_size_row, new_size_col, 3))
    batch_steering = np.zeros(batch_size)
    while 1:
        for i_batch in range(batch_size):
            i_line = np.random.randint(len(data))
            line_data = data.iloc[[i_line]].reset_index()
            keep_pr = 0
            x,y = preprocess_image_file_train(line_data)
#             while keep_pr == 0:
#                 x,y = preprocess_image_file_train(line_data)
#                 pr_unif = np.random
#                 if abs(y)<.1:
#                     pr_val = np.random.uniform()
#                     if pr_val>pr_threshold:
#                         keep_pr = 1
#                 else:
#                     keep_pr = 1
            
            x = x.reshape(1, x.shape[0], x.shape[1], x.shape[2])
            y = np.array([[y]])
            batch_images[i_batch] = x
            batch_steering[i_batch] = y
        yield batch_images, batch_steering

In [17]:
# # TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# model = Sequential()
# # model.add(Convolution2D(32, 2, 2, input_shape=(32, 32, 3)))
# # model.add(Reshape(input_shape + (1, ), input_shape=input_shape))

# # model.add(Convolution2D(32, 3,3, input_shape=(160, 320, 3)))
# model.add(Convolution2D(32, 3, 3,input_shape=(64, 64, 3)))
# model.add(Convolution2D(32, 3, 3))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Convolution2D(64, 3, 3))
# model.add(Convolution2D(64, 3, 3))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))


# model.add(Convolution2D(128, 3, 3))
# model.add(Convolution2D(128, 3, 3))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dense(16))

# # model.add(Dense(len(classes)))
# # model.add(Activation('relu'))
# model.add(Activation('softmax'))

# # model.add(Dense(1164))
# # # model.add(Dense(100))
# # # model.add(Dense(50))
# # # model.add(Dense(10))

# # model.add(Activation('relu'))
# # model.add(Dense(100))
# # model.add(Activation('relu'))
# # model.add(Dense(50))
# # model.add(Activation('relu'))
# # model.add(Dense(10))


In [ ]:

model.compile('adam',loss = 'mse',metrics = ['accuracy'])


# checkpointer = ModelCheckpoint(filepath="model.hdf5", verbose=1, save_best_only=True)

for i_pr in range(8):
#     print(df)
    train_r_generator = generate_train_from_PD_batch(df,batch_size)
#     gen(20, args.host, port=args.port),
    nb_vals = np.round(len(df)/val_size)-1
    model.fit_generator(
        train_r_generator,
        samples_per_epoch=20000,
        nb_epoch=10,
        verbose=1)
    
    pr_threashold = 1/(i_pr+1)*1

# history = model.fit(x_data, y_data, batch_size=128, nb_epoch=2, validation_split=0.2)
# def generate_arrays_from_file():
#     for i in range(0,len(y_data)):
#         yield (x_data[i],y_data[i])

# # #     for i in range(0,len(df)):
    

# model.fit_generator(
#     generate_arrays_from_file(),
#     samples_per_epoch=10000,
#     nb_epoch=1)
# assert(history.history['val_acc'][-1] > 0.91), "The validation accuracy is: %.3f.  It should be greater than 0.91" % history.history['val_acc'][-1]
print('Tests passed.')

Epoch 1/10
20224/20000 [==============================] - 86s - loss: 2.3559 - acc: 0.1542    

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1527: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '



Epoch 2/10
 3840/20000 [====>.........................] - ETA: 68s - loss: 0.1430 - acc: 0.2068

In [ ]:
import json
json_string = model.to_json()
model.save_weights("model.h5", True)
with open('model.json', 'w') as outfile :
    json.dump(json_string, outfile)


In [ ]:
print(df.iloc[[0]]['steering'][0])